# Data Streaming

## Kafka Streaming

### Topic Creation

In [4]:
from python_script.hdfs_commander import run_hdfs_command as hdfs

topics = ['nouns', 'verbs', 'functionWords', 'adverbs', 'adjectives']

for t in topics:
    hdfs(f'kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic {t}')

Output: Created topic nouns.

Output: Created topic verbs.

Output: Created topic functionWords.

Output: Created topic adverbs.

Output: Created topic adjectives.



### Topic Listing and Describing

In [5]:
print("List of Topics:")
hdfs('kafka-topics.sh --list --bootstrap-server localhost:9092')

print('-' * 150)
print("Description of Topics:")
for t in topics:
    hdfs(f'kafka-topics.sh --describe --bootstrap-server localhost:9092 --topic {t}')

List of Topics:
Output: adjectives
adverbs
functionWords
nouns
verbs

------------------------------------------------------------------------------------------------------------------------------------------------------
Description of Topics:
Output: Topic: nouns	TopicId: nqOFl_57RqOeDxelyiM3Aw	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: nouns	Partition: 0	Leader: 0	Replicas: 0	Isr: 0

Output: Topic: verbs	TopicId: QiBj1dgURTWd0xjFCLl5hA	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: verbs	Partition: 0	Leader: 0	Replicas: 0	Isr: 0

Output: Topic: functionWords	TopicId: m_xnnO3bSsatqT0wnOwoGQ	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: functionWords	Partition: 0	Leader: 0	Replicas: 0	Isr: 0

Output: Topic: adverbs	TopicId: Csco1UO1Q9S31DoM1MEQXw	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: adverbs	Partition: 0	Leader: 0	Replicas: 0	Isr: 0

Output: Topic: adjectives	TopicId: vjCeEEO-QDqj2-UPxa-9ew	PartitionCount: 1	ReplicationFactor: 1	Con

### Topic Deletion (run when necessary)

In [3]:
for t in topics:
    hdfs(f'kafka-topics.sh --delete --bootstrap-server localhost:9092 --topic {t}')

After topic creation, please proceed to the following steps:

1. open two different terminals.
2. change the directory to `~/DE-Assignment/shell_script` using `cd` command.
3. in one of the terminal, run the command `./consumers_starter.sh`.
4. in the another, run the command `./producers_starter.sh`.

## Spark Structured Streaming

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

24/12/18 21:14:26 WARN Utils: Your hostname, JORDAN. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/18 21:14:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/18 21:14:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
directory_path = "google_books_data/"

import os
if not os.path.isdir(directory_path):
    raise ValueError(f"The path {directory_path} is not a valid directory.")
    
csv_stream_df = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .schema("vocab STRING, word_length INT, part_of_speech STRING, \
            definitions STRING, sentences_eg STRING, derived_words STRING, synonyms STRING, \
            antonyms STRING, freq_of_usage INT, polarity STRING, subjectivity STRING, subjectivity_score STRING") \
    .option("basePath", directory_path) \
    .load(directory_path)

In [3]:
print("Streaming Topic: kata nama")
filtered_df = csv_stream_df.filter(csv_stream_df['part_of_speech'] == 'kata nama')

query = filtered_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

Streaming Topic: kata nama


24/12/18 21:14:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-33efe804-7426-4925-93d1-9264b31db098. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/18 21:14:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------+-------------+--------+--------------------+-------------------+
|    vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|antonyms|freq_of_usage|polarity|        subjectivity| subjectivity_score|
+---------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------+-------------+--------+--------------------+-------------------+
|     ilmu|          4|     kata nama|1. pengetahuan dl...|1. ilmu bumi;2. i...|    berilmu;keilmuan|                NULL|    NULL|            6|    none|partially fact-dr...|0.25297619047619047|
|      isi|          3|     kata nama|1. sesuatu yg ter...|1. apakah isi bak...|seisi;berisi;meng...|kandungan;muatan;.

In [4]:
print("Streaming Topic: kata kerja")
filtered_df = csv_stream_df.filter(csv_stream_df['part_of_speech'] == 'kata kerja')

query = filtered_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

Streaming Topic: kata kerja


24/12/18 21:14:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-51aae749-649e-48b5-a2f4-8f3c6eac347f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/18 21:14:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+--------+--------------------+------------------+
|      vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|        antonyms|freq_of_usage|polarity|        subjectivity|subjectivity_score|
+-----------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+--------+--------------------+------------------+
|pengetahuan|         11|    kata kerja|1. perihal menget...|1. kisah serong y...|tahu-tahu;mengeta...|maklum;faham;meng...|jahil;membiarkan|            3|    none|partially opinion...|0.6638888888888889|
+-----------+-----------+--------------+--------------------+--------------------+-

In [5]:
print("Streaming Topic: kata tugas")
filtered_df = csv_stream_df.filter(csv_stream_df['part_of_speech'] == 'kata tugas')

query = filtered_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

Streaming Topic: kata tugas


24/12/18 21:14:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-417b49bd-b73b-498b-b5ca-3b99c569a451. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/18 21:14:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------+-------------+--------+--------------------+-------------------+
|  vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|antonyms|freq_of_usage|polarity|        subjectivity| subjectivity_score|
+-------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------+-------------+--------+--------------------+-------------------+
| sahaja|          6|    kata tugas|1. yg satu-satuny...|1. tuan sahaja yg...|bersahaja;bersaha...|satu-satunya;cuma...|  jarang|           13|    none|partially fact-dr...|0.45833333333333337|
|  hanya|          5|    kata tugas|1. tidak lebih dr...|1. kemanakan hamb...|                NULL|                NULL|    NUL

In [6]:
print("Streaming Topic: adverba")
filtered_df = csv_stream_df.filter(csv_stream_df['part_of_speech'] == 'adverba')

query = filtered_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

Streaming Topic: adverba


24/12/18 21:14:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-76c9cc62-99db-4f7a-bba6-e19ed1ca8e1e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/18 21:14:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----------+--------------+-----------+------------+-------------+--------+--------+-------------+--------+------------+------------------+
|vocab|word_length|part_of_speech|definitions|sentences_eg|derived_words|synonyms|antonyms|freq_of_usage|polarity|subjectivity|subjectivity_score|
+-----+-----------+--------------+-----------+------------+-------------+--------+--------+-------------+--------+------------+------------------+
+-----+-----------+--------------+-----------+------------+-------------+--------+--------+-------------+--------+------------+------------------+



In [7]:
print("Streaming Topic: adjektif")
filtered_df = csv_stream_df.filter(csv_stream_df['part_of_speech'] == 'adjektif')

query = filtered_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

Streaming Topic: adjektif


24/12/18 21:14:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3901ce55-6140-4d63-826d-207fa37c1ba4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/18 21:14:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+------------------+
| vocab|word_length|part_of_speech|         definitions|        sentences_eg|       derived_words|            synonyms|            antonyms|freq_of_usage|polarity|        subjectivity|subjectivity_score|
+------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------------------+------------------+
|senang|          6|      adjektif|1. (serba) mudah,...|1. kerja mereka s...|bersenang;bersena...|mudah;gampang;rin...|susah;benci;sesak...|            0|positive|partially opinion...|0.7183333333333334|
| mudah|          5|      adjektif|1. tidak susah at...|1. sajak baru, bu...|bermudah-m

In [8]:
spark.stop()